In [5]:
import cv2
import mediapipe as mp
import numpy as np

video_path = 'POSE3.mp4'
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
current_frame_number = 0
paddle_states = []

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose_states = []

paddle_results = {}

output = cv2.VideoWriter('POSE3_output.mp4', -1, 12, (width, height))

for i in range(total_frames):
    paddle_results[i] = True

def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

def pose_analysis(frame):
    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5,
                            min_tracking_confidence=0.5) as pose:
        stance = "None"

        frame.flags.writeable = True
        # frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        # results = pose.process(frame)
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate angles
            angle_hipshoulderelbow_left = calculate_angle(hip_left, shoulder_left, elbow_left)
            angle_hipshoulderelbow_right = calculate_angle(hip_right, shoulder_right, elbow_right)

            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            if angle_hipshoulderelbow_left > 50:
                if angle_hipshoulderelbow_right < 45:
                    print(str(current_frame_number) + " Right hit")
                    stance = "Right hit"
                else:
                    stance = "Stance OK"
            if angle_hipshoulderelbow_right > 50:
                if angle_hipshoulderelbow_left < 45:
                    print(current_frame_number, "Left Hit")
                    stance = "Left hit"
                else:
                    stance = "Stance OK"

            cv2.putText(frame, str(stance),
                    tuple(np.multiply(shoulder_left, [width, height]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(frame, f"{str(int(angle_hipshoulderelbow_left))} - {str(int(angle_hipshoulderelbow_right))}",
                    tuple(np.multiply(shoulder_right, [width, height]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(frame, get_timecode(current_frame_number), (25, 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print("Exception: ", e)
            pass


    # if self.current_frame_number > 400:
    # cv2.imshow("Frame", frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    output.write(frame)
    return stance

def check_for_invalid_pose(previous_states: list) -> bool:
    for last_state in previous_states:
        if "None" in last_state["stance"]:
            return False
    return True

def was_green_light_on(frame_number: int, paddle_results: dict) -> bool:
    if frame_number < 30:
        return False
    for i in range(7):
        if paddle_results[frame_number - (i * 5)]:
            return True
    return False

def get_timecode(current_frame: int) -> str:
    if fps == 0:
        return "00:00:00"

    # Calculate the current time in seconds
    current_time_in_seconds = current_frame / fps

    # Calculate hours, minutes, seconds and frames
    minutes, seconds = divmod(current_time_in_seconds, 60)
    hours, minutes = divmod(minutes, 60)
    frames = current_frame - int(current_time_in_seconds) * fps

    # Format timecode as HH:MM:SS:FF
    timecode = "%02d:%02d:%02d" % (minutes, seconds, frames)

    return timecode

def evaluate_results(paddle_results: dict):
    results = []
    last_result = {"frame": -999, "stance": "None"}
    for i, state in enumerate(pose_states):
        if i < 5:
            continue
        if "hit" not in state["stance"]:
            continue

        diff = state["frame"] - last_result["frame"]
        if diff < fps:
            print("difference too low: ", diff)
            continue

        if not check_for_invalid_pose(pose_states[i - 3:i]):
            print("None in last 3: ", get_timecode(state["frame"]))
            continue

        if i < len(pose_states) - 3:
            if not check_for_invalid_pose(pose_states[i + 1:i + 3]):
                print("None in next 3: ", get_timecode(state["frame"]))
                continue

        result = state["stance"]
        if not was_green_light_on(state["frame"], paddle_results):
            print(state["frame"], "invalid hit!")
            result = "Invalid " + result

        results.append({"time_code": get_timecode(state["frame"]), "result": result})
        last_result = state

    return results

while cap.isOpened():
    ret, frame = cap.read()

    if not ret or frame is None or frame.shape[0] == 0 or frame.shape[1] == 0:
        break

    # Process every 5 frames
    if current_frame_number % 5 == 0:
        pose_result = pose_analysis(frame)
        pose_states.append({"frame": current_frame_number, "stance": pose_result})

    current_frame_number += 1

results = evaluate_results(paddle_results)

cap.release()
output.release()
cv2.destroyAllWindows()

Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exception:  get_timecode() missing 1 required positional argument: 'current_frame'
Exce